# Capstone One Data Wrangling

In [239]:
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [240]:
# pandas options
pd.set_option('display.max_columns', 125) # csv contains 124 columns
pd.set_option('display.max_rows', 1000) # display more rows
pd.options.mode.chained_assignment = None  # turn off false positive warnings for editing a copy

In [241]:
data = pd.read_csv('montereyClimateData.csv') # had one float at end of this col

Get relevant columns from the data set:

In [242]:
df = data
columns = ['DATE', 'HourlyPresentWeatherType', 'HourlySkyConditions', 'HourlyVisibility', 'HourlyDryBulbTemperature', 'HourlyWindSpeed', 'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 'DailyPeakWindSpeed', 'DailyPrecipitation']
df = df[columns]

### 1. Convert the 'DATE' column into a new datetime column full of datetime objects, and set this new column as the dataframe's index.

In [243]:
def date_val_to_datetime(to_parse):
    to_format = to_parse.split('T')
    return datetime.strptime(to_format[0] + ' ' + to_format[1], '%Y-%m-%d %H:%M:%S')

df['datetime'] = df['DATE'].apply(date_val_to_datetime)

### 2. Set this new column to be a datetime index for the dataframe.

In [244]:
df = df.set_index(['datetime'])

### 3. Backfill the maximum daily temperature, minimum daily temperature, daily precipitation, and maximum daily wind speed columns throughout the day. 


In [245]:
df[['DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 'DailyPeakWindSpeed', 'DailyPrecipitation']] = df[['DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 'DailyPeakWindSpeed', 'DailyPrecipitation']].bfill()

### 4. Converted daily measurements from string values to float values.

In [246]:
# sometimes these string values come with an 's' suffix -> taking all but the last char to convert
df['DailyMaximumDryBulbTemperature'] = df['DailyMaximumDryBulbTemperature'].str.replace('s','').astype(float)
df['DailyMinimumDryBulbTemperature'] = df['DailyMinimumDryBulbTemperature'].str.replace('s','').astype(float)
# df['DailyMinimumDryBulbTemperature'] = df['DailyMinimumDryBulbTemperature'].str[:-1].astype(float)

### 5. Replace daily precipitation column’s “T and “Ts” values, which indicate “traces of precipitation,” with zeros.


In [247]:
def precip_str_to_float(value):
    """Checks for string or float and returns appropriate float value"""
    if isinstance(value, str):
        # second conditional catches two cases: some values are 'Ts' and others are '0.00s' strings
        if value is 'T' or value[-1] is 's': 
            return 0.0
        else:
            return float(value)
    else:
        return value

df['DailyPrecipitation'] = df['DailyPrecipitation'].apply(precip_str_to_float)

### 6. Replace sky condition string list with a list of dictionaries. The sky condition is the key and a SkyCondition namedtuple is the value.

In [248]:
# column value is a string of a list of codes, 'BKN:07 15 OVC:08 20'
# desired output is a list of tuples, [('BKN', 7, 15), ('OVC', 8, 20)]
# clear days lack a second integer, i.e., 'CLR:00', appending 0 in place of missing value
from collections import namedtuple

SkyCondition = namedtuple('SkyCondition', 'obscuration, vertical_distance') # these will be the dict's values

def list_of_lists_by_n(the_list, n):
    """Yields the next n elements of a list as a sublist"""
    for i in range(0, len(the_list), n):  
        yield the_list[i:i + n] 
        
def from_many_to_two(the_string):
    split_at_spaces = the_string.split(' ')
    return list(list_of_lists_by_n(split_at_spaces, 2))

def from_two_to_three(list_of_lists):
    """
    input: ['CAPS:02', '35']
    output: {'CAPS':, SkyCondition(obscuration=02, vertical_distance=35)}
    """
    output = []
    for two_element_list in list_of_lists:
        first_element = two_element_list[0]
        if 2 >= len(first_element):
            return {} # for single trailing ints
        first_element_split = first_element.split(":")
        if 2 > len(two_element_list): 
            two_element_list.append(0) # catch CLR days missing following 00
        condition = SkyCondition(int(first_element_split[1]), int(two_element_list[1]))
        output.append({first_element_split[0]: condition})
    return output

def condition_string_to_namedtuple_dict(value):
    """
    Converts string containing several of the following to a list of dictionaries as follows:
    input: "CAPS:03 34"
    output: {'CAPS':, SkyCondition(obscuration=3, vertical_distance=34)}
    """
    if isinstance(value, float): # the only floats are np.nan, which is a float...with a str repr
        return [] # replace NaNs as an empty list
    the_string = value
    list_of_twos = from_many_to_two(the_string)
    return from_two_to_three(list_of_twos)

df['HourlySkyConditions'] = df['HourlySkyConditions'].apply(condition_string_to_namedtuple_dict)

In [249]:
# example list comprehension for polling for sky condition in particular hour
print(df['HourlySkyConditions'][212])
['BKN' in x for x in df['HourlySkyConditions'][212]]

[{'FEW': SkyCondition(obscuration=2, vertical_distance=19)}, {'BKN': SkyCondition(obscuration=7, vertical_distance=47)}, {'OVC': SkyCondition(obscuration=8, vertical_distance=55)}]


[False, True, False]

In [250]:
# example lookup and average to calculate obscuration score for an hour
os = [[y.obscuration for x, y in d.items()] for d in df['HourlySkyConditions'][212]]
o_score = sum([x[0] for x in os]) / len(os) # average obscuration score for the hour
print(o_score)
# But perhaps it's more significant that, in this particular hour, the sky went from sunny to overcast....

5.666666666666667
